# Notebook to calculate zipcodes based on lat-lon coordinates

In [216]:
import numpy as np
import pandas as pd
import sys
import json
import os
import shapely
import shapely.geometry
from shapely.geometry import Point, Polygon, MultiPolygon, shape
import urllib.request as ulr
from googleplaces import GooglePlaces, types, lang
YOUR_API_KEY = os.environ['GOOGLE_API_KEY']
#YOUR_API_KEY = <Put your GooglePlaces-enabled Google API Key here>

In [7]:
pd.set_option('max_rows', 10)

## Load crime lat-lon data

In [8]:
crime_latlon_df = pd.read_csv('crime_latlon.out', header=None)
crime_latlon_df[0] = pd.to_numeric(crime_latlon_df[0].str.strip(to_strip = '('))
crime_latlon_df[1] = crime_latlon_df[1].str.strip(to_strip = ')').str.strip()
crime_latlon_df[1] = pd.to_numeric(crime_latlon_df[1])
crime_latlon_df[2] = pd.to_numeric(crime_latlon_df[2])
crime_latlon_df = crime_latlon_df.dropna(axis=0)
crime_latlon_df = crime_latlon_df.rename(index=str, columns={0:'lat',1:'lon',2:'crimes'})
crime_latlon_df.head()

,lat,lon,crimes
0,40.675955,-73.735304,5.0
1,40.804214,-73.964751,188.0
2,40.670460,-73.882559,65.0
3,40.822017,-73.938915,266.0
4,40.749871,-73.898905,63.0


## Match zipcodes based on shapefile (Fastest)

### Load in shapefile of NYC Zip Code geometries

In [ ]:
import json

with open('NYC_zipcodes.geojson') as f:
    data = json.load(f)
polys = []
for i in range(len(data['features'])):
    polys.append(Polygon(data['features'][i]['geometry']['coordinates'][0]))

In [ ]:
geo_lat = []
geo_lon = []
geo_zip = []
search_failures_geo = []
failure_index_geo = []
ct_geo = 0

In [ ]:
from tqdm import tqdm 
import time

#Adjust the range as needed for missing values
for i in tqdm(np.arange(start_index,end_index)):
    try:
        success = 0
        lat = crime_latlon_df['lat'][i]
        lon = crime_latlon_df['lon'][i]
        pt = Point(crime_latlon_df['lon'][i],crime_latlon_df['lat'][i])
        for j in range(len(data['features'])):
            if polys[j].contains(pt):
                geo_zip.append(data['features'][j]['properties']['postalCode'])
                success = 1
                break
        if success == 0:
            geo_zip.append('ZIP_NOT_FOUND')
        geo_lat.append(lat)
        geo_lon.append(lon)
        

    except:
        ct = ct + 1
        geo_lat.append(crime_latlon_df['lat'][i])
        geo_lon.append(crime_latlon_df['lon'][i])
        geo_zip.append('SEARCH FAILURE')
        search_failures_geo.append((crime_latlon_df['lat'][i],crime_latlon_df['lon'][i]))
        failure_index_geo.append(i)
        if np.mod(ct_geo,25) == 0:
            print('Failure ' + str(ct_geo) + ': ' + str((crime_latlon_df['lat'][i],crime_latlon_df['lon'][i])))

   

In [ ]:
df_zips = pd.DataFrame([geo_lat, geo_lon, geo_zip]).T
df_zips = df_zips.rename(index=str, columns={0: 'lat', 1: 'lon', 2: 'zipcode'})
df_merged = df_zips.merge(crime_latlon_df, on=['lat','lon'])

df_merged

In [ ]:
out_file_name = 'crime_latlon_zips.csv'
df_merged.to_csv(out_file_name)

### Alternative Method: GooglePlaces API

In [112]:
#Grabbing and parsing the JSON data
def GoogPlac(lat,lng,radius,key):
    #making the url
    AUTH_KEY = key
    LOCATION = str(lat) + "," + str(lng)
    RADIUS = radius
    MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
             '?location=%s'
             '&radius=%s'
             #'&types=%s'
             '&sensor=false&key=%s') % (LOCATION, RADIUS,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data

def GoogGeocode(lat,lng,key):
    #making the url
    AUTH_KEY = key
    url = ("https://maps.googleapis.com/maps/api/geocode/json?latlng=" + str(lat) +','+ str(lng) + '&key=' + AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(url)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data

def GoogPlacText(loc,radius,key):
    #making the url
    AUTH_KEY = key
    LOCATION = loc.replace(' ', '+')
    RADIUS = radius
    MyUrl = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
             '?query=%s'
             '&radius=%s'
             '&sensor=false&key=%s') % (LOCATION, RADIUS,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data

def GoogPlacID(ID,key):
    AUTH_KEY = key
    PLACE_ID = ID
    MyUrl = ('https://maps.googleapis.com/maps/api/place/details/json'
             '?placeid=%s'
             '&key=%s') % (PLACE_ID,AUTH_KEY)
    #grabbing the JSON result
    response = ulr.urlopen(MyUrl)
    data = response.read().decode("utf-8")
    data = json.loads(data)
    return data


### Test out geocoding on known address (NYU Stern, zipcode 10012)

In [9]:
lat,lon = 40.729242, -73.996491
query = GoogPlac(lat,lon, 10, YOUR_API_KEY)
query2 = GoogPlacID(query['results'][1]['place_id'],YOUR_API_KEY)
for i in range(len(query2['result']['address_components'])):
    if query2['result']['address_components'][i]['types'] == ['postal_code']:
        print(query2['result']['address_components'][i]['long_name'])

10012


### Test out geocoding on known address (NYU Tandon, zipcode 11201)

In [33]:
lat,lon = 40.694187, -73.986558
query = GoogPlac(lat,lon, 10, YOUR_API_KEY)
query2 = GoogPlacID(query['results'][1]['place_id'],YOUR_API_KEY)
for i in range(len(query2['result']['address_components'])):
    if query2['result']['address_components'][i]['types'] == ['postal_code']:
        print(query2['result']['address_components'][i]['long_name'])

11201


### Test out geocoding on known address (Brunswick, ME, zipcode 04011)

In [35]:
lat,lon = 43.899295, -69.964007
query = GoogPlac(lat,lon, 10, YOUR_API_KEY)
query2 = GoogPlacID(query['results'][0]['place_id'],YOUR_API_KEY)
for i in range(len(query2['result']['address_components'])):
    if query2['result']['address_components'][i]['types'] == ['postal_code']:
        print(query2['result']['address_components'][i]['long_name'])

04011


If the location is a dense area, the first hit of the google search is likely the administrative area (multiple zipcodes). If not, the second hit likely will be. We can check both entries and stop once we have a zip code.

Initialize our storage lists:

In [10]:
pl_lat = []
pl_lon = []
pl_zip = []
search_failures = []
failure_index = []
ct = 0

Specify our index ranges:

In [83]:
start_index = 0
end_index = len(crime_latlon_df)

In [32]:
from tqdm import tqdm 
import time

#Adjust the range as needed for missing values
for i in tqdm(np.arange(start_index,end_index)):
    try:
        success = 0
        lat = crime_latlon_df['lat'][i]
        lon = crime_latlon_df['lon'][i]
        query = GoogPlac(lat,lon, 10, YOUR_API_KEY)
        for j in range(2):
            if success == 0:
                query2 = GoogPlacID(query['results'][j]['place_id'],YOUR_API_KEY)
                for i in range(len(query2['result']['address_components'])):
                    if query2['result']['address_components'][i]['types'] == ['postal_code']:
                        pl_zip.append(query2['result']['address_components'][i]['long_name'])
                        success = 1
                        break
        if success == 0:
            pl_zip.append('ZIP_NOT_FOUND')
        pl_lat.append(lat)
        pl_lon.append(lon)


    except:
        ct = ct + 1
        pl_lat.append(crime_latlon_df['lat'][i])
        pl_lon.append(crime_latlon_df['lon'][i])
        pl_zip.append('SEARCH FAILURE')
        search_failures.append((crime_latlon_df['lat'][i],crime_latlon_df['lon'][i]))
        failure_index.append(i)
        if np.mod(ct,25) == 0:
            print('Failure ' + str(ct) + ': ' + str((crime_latlon_df['lat'][i],crime_latlon_df['lon'][i])))

   

 11%|█         | 4615/42826 [32:04<6:54:45,  1.54it/s]

Failure 425: (40.588189569999997, -73.962251093000006)


 25%|██▌       | 10878/42826 [1:15:18<6:28:51,  1.37it/s]

Failure 450: (40.844945251999995, -73.883956741999995)


 37%|███▋      | 15664/42826 [1:49:42<6:03:01,  1.25it/s]

...

 87%|████████▋ | 37253/42826 [4:58:40<01:19, 69.99it/s]

Failure 925: (40.590812788000001, -73.926539289999994)


 87%|████████▋ | 37261/42826 [4:58:40<01:23, 66.84it/s]

Failure 950: (40.755299764, -73.908465863999993)


100%|██████████| 42826/42826 [5:25:47<00:00,  3.70it/s]


In [39]:
print(len(pl_lat))
print(len(pl_lon))
print(len(pl_zip))
print(len(search_failures))
print(len(failure_index))

112826
112826
112826
958
958


### Rerun any latlon pairs that failed (due to internet connectivity)

In [46]:
pl_lat_fail = []
pl_lon_fail = []
pl_zip_fail = []
search_failures_2 = []
failure_index_2 = []
ct_fail = 0

In [47]:
from tqdm import tqdm 
import time

#Adjust the range as needed for missing values
for i in tqdm(np.arange(len(failure_index))):
    try:
        success = 0
        lat = pl_lat[failure_index[i]]
        lon = pl_lon[failure_index[i]]
        query = GoogPlac(lat,lon, 10, YOUR_API_KEY)
        for j in range(2):
            if success == 0:
                query2 = GoogPlacID(query['results'][j]['place_id'],YOUR_API_KEY)
                for i in range(len(query2['result']['address_components'])):
                    if query2['result']['address_components'][i]['types'] == ['postal_code']:
                        pl_zip_fail.append(query2['result']['address_components'][i]['long_name'])
                        success = 1
                        break
        if success == 0:
            pl_zip_fail.append('ZIP_NOT_FOUND')
        pl_lat_fail.append(lat)
        pl_lon_fail.append(lon)


    except:
        ct_fail = ct_fail + 1
        pl_lat_fail.append(pl_lat[failure_index[i]])
        pl_lon_fail.append(pl_lat[failure_index[i]])
        pl_zip_fail.append('SEARCH FAILURE 2')
        search_failures_2.append((pl_lat[failure_index[i]],pl_lon[failure_index[i]]))
        failure_index_2.append(i)
        if np.mod(ct_fail,25) == 0:
            print('Failure ' + str(ct_fail) + ': ' + str((pl_lat[failure_index[i]],pl_lon[failure_index[i]])))

   

100%|██████████| 958/958 [06:16<00:00,  2.46it/s]


Add in the re-run zipcodes

In [52]:
for i in range(len(failure_index)):
    pl_zip[failure_index[i]] = pl_zip_fail[i]

Merge our (partial) zipcode information with the (partial) crime data

In [53]:
df_zips = pd.DataFrame([pl_lat, pl_lon, pl_zip]).T
df_zips = df_zips.rename(index=str, columns={0: 'lat', 1: 'lon', 2: 'zipcode'})
#df_merged = df_zips.merge(crime_latlon_df.iloc[start_index:end_index], on=['lat','lon'])
df_merged = df_zips.merge(crime_latlon_df, on=['lat','lon'])

df_merged

,lat,lon,zipcode,crimes
0,40.676,-73.7353,11422,5.0
1,40.676,-73.7353,SEARCH FAILURE,5.0
2,40.8042,-73.9648,10025,188.0
3,40.6705,-73.8826,11208,65.0
4,40.822,-73.9389,10039,266.0
...,...,...,...,...
112821,40.6475,-73.9803,11218,1.0
112822,40.6076,-74.1358,10314,1.0
112823,40.6166,-73.9745,11230,1.0
112824,40.6768,-73.9604,11238,1.0


Aggregate crime counts at the zipcode level

In [238]:
df_merged.groupby('zipcode').sum()

,crimes
zipcode,
00083,4834.0
10001,75590.0
10002,52297.0
10003,54137.0
10004,4323.0
...,...
11692,9815.0
11693,8895.0
11694,7797.0


Save our (partial) lat-lon-zip data

In [ ]:
out_file_name = 'googplac_partial_crime_latlon_zips_' + str(start_index) + '_' + str(end_index-1) + '.csv'
#out_file_name = 'googplac_crime_latlon_zips.csv'

df_merged.to_csv(out_file_name)

## Load Subway CSV

In [ ]:
subway_df = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv', header=0)
subway_station_df = subway_df.groupby('Station Name').first()[['Station Latitude', 'Station Longitude', 'Station Location', 'Entrance Location']]

### Match zipcodes to latlon using Google Geocode API (only 356 of them)

In [117]:
a = (subway_station_df[['Station Latitude', 
                'Station Longitude']].values.astype(float))
len(a)
zipcodesDF = pd.DataFrame()
subway_station_df['zipcodes'] = np.zeros(len(subway_station_df))
for latlon in pd.DataFrame(a).drop_duplicates().values:
    query = GoogGeocode(latlon[0],latlon[1], YOUR_API_KEY)
    revgeo = query["results"][0]['address_components'][-1]
    #print (revgeo)
    subway_station_df['zipcodes'][(subway_station_df['Station Latitude'] == latlon[0]) * 
           (subway_station_df['Station Longitude'] == latlon[1])] = revgeo['long_name']
subway_station_df.head()

//anaconda/envs/mypy35/lib/python3.5/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
//anaconda/envs/mypy35/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Station Latitude,Station Longitude,Station Location,Entrance Location,zipcodes
Station Name,,,,,
103rd St,40.796092,-73.961454,"(40.796092, -73.961454)","(40.796263, -73.961610)",10025
104th St-102nd St,40.695178,-73.844330,"(40.695178, -73.84433)","(40.695033, -73.843298)",11418
104th St-Oxford Av,40.681711,-73.837683,"(40.681711, -73.837683)","(40.681994, -73.836742)",11417
110th St,40.795020,-73.944250,"(40.79502, -73.94425)","(40.794804, -73.944613)",3243
110th St-Central Park North,40.799075,-73.951822,"(40.799075, -73.951822)","(40.798497, -73.952672)",10026


### Reinstate the leading 0 that was deleted from some zipcodes

In [131]:
for i in range(len(subway_station_df)):
    if len(subway_station_df['zipcodes'][i]) == 4:
        subway_station_df['zipcodes'][i] = '0' + subway_station_df['zipcodes'][i]


//anaconda/envs/mypy35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [210]:
subway_station_df[subway_station_df['zipcodes'] == '11373']

,Station Latitude,Station Longitude,Station Location,Entrance Location,zipcodes
Station Name,,,,,
82nd St-Jackson Heights,40.747659,-73.883697,"(40.747659, -73.883697)","(40.747812, -73.883876)",11373
Elmhurst Av,40.742454,-73.882017,"(40.742454, -73.882017)","(40.741729, -73.881135)",11373
Jackson Heights-Roosevelt Ave,40.746644,-73.891338,"(40.746644, -73.891338)","(40.746784, -73.890958)",11373


In [133]:
subway_station_df

,Station Latitude,Station Longitude,Station Location,Entrance Location,zipcodes
Station Name,,,,,
103rd St,40.796092,-73.961454,"(40.796092, -73.961454)","(40.796263, -73.961610)",10025
104th St-102nd St,40.695178,-73.844330,"(40.695178, -73.84433)","(40.695033, -73.843298)",11418
104th St-Oxford Av,40.681711,-73.837683,"(40.681711, -73.837683)","(40.681994, -73.836742)",11417
110th St,40.795020,-73.944250,"(40.79502, -73.94425)","(40.794804, -73.944613)",03243
110th St-Central Park North,40.799075,-73.951822,"(40.799075, -73.951822)","(40.798497, -73.952672)",10026
...,...,...,...,...,...
Woodside Av-61st St,40.745630,-73.902984,"(40.74563, -73.902984)","(40.745531, -73.903248)",11377
World Trade Center,40.712582,-74.009781,"(40.712582, -74.009781)","(40.713333, -74.009033)",10007
Yankee Stadium-161st St,40.827905,-73.925651,"(40.827905, -73.925651)","(40.827840, -73.925715)",10452


### Save our results to CSV